# Internal Coordinates and the B-matrix
## Defining bonds
Assuming a reasonable initial geometrical input, the identification of which atoms are bonded may, in most cases, be simply achieved using standard covalent radii.  Here we do this for ammonium.

In [1]:
import psi4
import numpy as np
mol = psi4.geometry("""
 1 1
  pubchem:ammonium
""")
mol.update_geometry()
xyzGeom = np.array(mol.geometry())
print ("Starting Geometry for Ammonium")
print (xyzGeom) 

# Get atomic numbers and covalent radii
import sys,os
sys.path.append('os.getcwd()')
from opt_helper import covRadii, v3d
from itertools import combinations, permutations
from math import sqrt
Natom = mol.natom()
Z = [int(mol.Z(i)) for i in range(Natom)]

# Connectivity matrix indicates which atoms are bonded.
C = np.zeros( (Natom, Natom), bool )
for i,j in combinations( range(Natom), 2):
  R = v3d.dist( xyzGeom[i], xyzGeom[j])
  Rcov = (covRadii.R[Z[i]] + covRadii.R[Z[j]])/0.52917720859
  if R < 1.3 * Rcov:
    C[i,j] = C[j,i] = True

print "Connectivity matrix for Ammonium"
print (C)

	Searching PubChem database for ammonium
	Found 1 result
Starting Geometry for Ammonium
[[  0.00000000e+00  -8.29781462e-35   3.92826956e-06]
 [  1.58424594e+00   4.85035429e-17   1.12021175e+00]
 [  4.85021276e-17  -1.58419971e+00  -1.12023904e+00]
 [ -1.58424594e+00  -4.85035429e-17   1.12021175e+00]
 [ -4.85021276e-17   1.58419971e+00  -1.12023904e+00]]
Connectivity matrix for Ammonium
[[False  True  True  True  True]
 [ True False False False False]
 [ True False False False False]
 [ True False False False False]
 [ True False False False False]]


In practice, the factor 1.3 may be increased as necessary to ensure bonds connect all the atoms together.  This is done in so-called 'supermolecule' approaches, even when multiple molecules are present.

## Determining internal coordinates automatically
Once the connectivity is determined, the chemically relevant bonds, angles, and dihedral angles can be determined from the connected atoms.  A stretching coordinate is added for every two bonded atoms, and a bending coordinate is added for all unique A-B-C sequences of connected atoms. For this example of ammonium, there are no dihedral angles.  However, if present, any bonded sequence A-B-C-D is taken to define a dihedral coordinate that is added to the set of optimization coordinates.  (If linear or near-linear bends are present, then 2 orthogonal bends are used to describe the situation, and dihedral coordinates which contain these linear bends must be avoided.  This complication is not further explored here.)

In [2]:
intcos = []
# Add stretches by connectivity.
from opt_helper import stre, bend
for i,j in combinations( range(Natom), 2 ):
  if C[i,j]:
    s = stre.STRE(i, j)
    if s not in intcos:
      intcos.append(s)

# Add bends by connectivity.
for i,j in permutations( range(Natom), 2):
  if C[i,j]:
    for k in range(i+1,Natom):  # make i<k
      if C[j,k]:
        (check, val) = v3d.angle(xyzGeom[i], xyzGeom[j], xyzGeom[k])
        if not check: continue
        b = bend.BEND(i, j, k)
        if b not in intcos:
          intcos.append(b)
          
# print our internal coordinate set with values
for i in intcos:
    print "%10s = %10.5f" % (i, i.q(xyzGeom))

    R(1,2) =    1.94028
    R(1,3) =    1.94027
    R(1,4) =    1.94028
    R(1,5) =    1.94027
  B(2,1,3) =    1.91064
  B(2,1,4) =    1.91065
  B(2,1,5) =    1.91064
  B(3,1,4) =    1.91064
  B(3,1,5) =    1.91060
  B(4,1,5) =    1.91064


## Redundant Coordinates
These simple internal coordinates have many advantages. They tend to produce a strongly diagonal energy second derivative matrix (Hessian), and they have force constants which are readily guessable, i.e., transferable from one molecule to another. In these two respects, they are vastly superior to Cartesian coordinates.

For many years, the primary difficulty was that the simple procedure demonstrated here generates too many coordinates. Non-linear molecules have $3N-6$ internal degrees of freedom (where $N$ is the number of atoms), but including all the internal coordinates defined by the bond connectivity will generally produce more than that.

For ammonium, we see that we have 10 coordinates, while the ion has only 9 internal degrees of freedom. Various alternative coordinate systems were explored, for example "delocalized coordinates" [J.Baker, A. Kessi, and B. Delley, _J. Chem. Phys._, 105, 192 (1996)], and "natural internal coordinates" [G. Fogarasi, X. Zhou, P.W. Taylor, and P. Pulay, _J. Am. Chem. Soc._, 114, 8191 (1992)].  However, an elegant and robust solution turned out to be simply to work with a redundant set and to make the necessary mathematical adjustments.  This approach was advanced by P. Pulay and G. Fogarasi [_J. Chem. Phys._, 96, 2856 (1992)].

## The B matrix and coordinate transformations

The Wilson __B__-matrix [see the classic text by E.B. Wilson, J.C. Decius, and P.C. Cross, _Molecular Vibrations_, Dover (1955)] is the key to transformations between internal and Cartesian coordinates.  A __B__-matrix element is defined as the derivative of an internal coordinate value with respect to a Cartesian coordinate.

$$\textbf {B}_{ij} = \frac{\delta q_i}{\delta x_j}$$

The __B__-matrix therefore has dimensions of (Number of internals) by (Number of Cartesians).  A row of the __B__-matrix (corresponding to a particular internal coordinate) may also be interpreted as the collection of Cartesian unit vectors for each atom along which displacing would result in a maximal increase in the value of the internal coordinate.  (These unit vectors are sometimes called "_s_-vectors".)  Formulas for these __B__-matrix components as well as their derivatives are provided in a 2002 paper of V. Bakken and T. Helgaker [_J. Chem. Phys._, 117, 9160 (2002)], albeit not without some errors in the dihedral derivative __B__-matrix formulas.

Lets take a look at the first row of the __B__-matrix for ammonia:

In [3]:
from opt_helper import intcosMisc
Bmat = intcosMisc.Bmat(intcos, xyzGeom)
print('B matrix - first row')
print(Bmat[0])

B matrix - first row
[ -8.16502224e-01  -2.49981709e-17  -5.77342289e-01   8.16502224e-01
   2.49981709e-17   5.77342289e-01   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00]


Notice that only the first 6 elements corresponding to the Cartesian coordinates of the first two atoms are non-zero. Since the first internal is R(1,2), only the first 2 atoms are relevant to its definition.  The B-matrix, therefore, is tremendously sparse, when the internal coordinates are restricted to stretches, bends, and dihedrals. (For Cartesian coordinates themselves, the __B__-matrix is simply the unit matrix.)

For a diatomic, the __B__-matrix consists of a single row, and the components are simply two vectors pointing outward as shown below.

In [4]:
H2mol = psi4.geometry("""
H
H 1 0.8
""")
H2intcos = [ stre.STRE(0,1) ]
H2mol.update_geometry()
H2xyzGeom = np.array(H2mol.geometry())
H2Bmat = intcosMisc.Bmat(H2intcos, H2xyzGeom)
print("B matrix for hydrogen molecule:")
print(H2Bmat)


B matrix for hydrogen molecule:
[[-0. -0. -1.  0.  0.  1.]]


The __B__-matrix is immediately useful for transforming small displacements in Cartesian coordinates into displacements in internal coordinates.

$$\textbf {B}_{ij} \delta x_j = \delta q_i$$

or in matrix-vector form

$$\textbf {B} \Delta x = \Delta q $$

However, since the __B__-matrix depends upon the geometry, this transformation is exact only for infinitesimal displacements.

## Transformations between internal and Cartesian forces
The __B__-matrix readily facilitates the transformation of the energy gradient into Cartesian coordinates from internal coordinates.

$$ \frac{\delta E}{\delta q_i} \frac{\delta q_i}{\delta x_j} = \frac{\delta E}{\delta x_j}$$
or
$$ \textbf{B}^T g_q = g_x $$ 

The reverse transformation is of equal practical importance. Quantum chemistry codes typically provide an energy gradient in Cartesian coordinates.  This gradient must be transformed into internal coordinates, in preparation for whatever step-determining algorithm (Newton-Raphson, etc.) will be used. This transformation is more complicated because 1) we need to invert the non-square B-matrix; and 2) the B-matrix is often redundant, i.e., the rows are linearly dependent.

The left, generalized inverse of $\textbf {B}^T$ we will call the $\textbf A^T$ matrix.  $\textbf A$ has dimension (Num. of Cartesians) by (Num. of internals) and is defined by

$$ \textbf A^T = (\textbf{B} \textbf{u} \textbf {B}^T)^{-1} \textbf {B} \textbf{u}$$

where __u__ is an arbitrary matrix of dimension (Num. of Cartesians) by (Num. of Cartesians).

That $\textbf A^T$ is the left inverse of $\textbf {B}^T$ may be readily seen by the following:

\begin{align}
 \textbf{B}^T g_q &= g_x \\
 \textbf{A}^T \textbf{B}^T g_q &= \textbf{A}^T g_x \\
 [ (\textbf{B} \textbf{u} \textbf{B}^T )^{-1} \textbf{B} \textbf{u} ] \textbf{B}^T g_q &= \textbf{A}^T g_x  \\
(\textbf{B} \textbf{u} \textbf{B}^T )^{-1} ( \textbf{B} \textbf{u} \textbf{B}^T ) g_q &= \textbf{A}^T g_x  \\
 g_q  &= (\textbf{B} \textbf{u} \textbf{B}^T )^{-1} \textbf{B} \textbf{u} g_x  
\end{align}




The first step in geometry optimization after the internal coordinates have been determined and a Cartesian energy derivative has been computed is to use this expression to compute the gradient in internal coordinates.  (One can instead think in terms of "forces" - the only difference is a minus sign.)  

For transforming energy derivatives, the choice of $\textbf{u}$ is irrelevant.  In some spectroscopic applications, it is comprised of inverse atomic masses on its diagonal.  For a thorough exposition on geometric derivative transformaions see the paper by W.D. Allen and A.G. Csaszar [_J. Chem. Phys._, 98, 2983 (1993)].

If $\textbf{u}$ is chosen to be the unit matrix, then the transformation reduces to

$$ g_q  = (\textbf{B} \textbf{B}^T )^{-1} \textbf{B} g_x $$

The matrix $\textbf{B} \textbf{B}^T$ will be symmetric.  If redundant coordinates are present, then the generalized inverse (or pseudo-inverse) must be computed.  The matrix is diagonalized, then the non-zero eigenvalues on the diagonal are inverted, and the matrix is transformed back. Below is an example of the transformation of the gradient from Cartesians to internals for ammonium.  The symmMatInv() function shows how this generalized inverse can be performed.

In [5]:
mol = psi4.geometry("""
1 1
  pubchem:ammonium
""")

mol.update_geometry()
psi4.set_options({'basis': 'dz'})
psi4gradientMatrix = psi4.gradient('scf')
gx = np.reshape( np.array(psi4gradientMatrix), (3*Natom))
# Removing some noise
from math import fabs
gx[:] = [0 if fabs(x) < 1.0e-16 else x for x in gx]
print ("Gradient in Cartesian coordinates")
print ( gx )

# Function to return a generalized inverse
def symmMatInv(A):
    dim = A.shape[0]
    det = 1.0

    evals, evects = np.linalg.eigh(A)
    evects = evects.T
    for i in range(dim):
        det *= evals[i]

    diagInv = np.zeros( (dim,dim), float)
    for i in range(dim):
        if fabs(evals[i]) > 1.0e-10:
            diagInv[i,i] = 1.0/evals[i]
            
    # A^-1 = P^t D^-1 P
    tmpMat = np.dot(diagInv, evects)
    AInv = np.dot(evects.T, tmpMat)
    return AInv

B = intcosMisc.Bmat(intcos, xyzGeom)
temp_arr = np.dot(B,gx.T)
G    = np.dot(B, B.T)
Ginv = symmMatInv(G)
gq = np.dot(Ginv,temp_arr.T)

print ("Gradient in Internal coordinates.")
print ( gq )

	Searching PubChem database for ammonium
	Found 1 result
Gradient in Cartesian coordinates
[  0.00000000e+00   0.00000000e+00  -6.59135992e-07   1.04753805e-02
   0.00000000e+00   7.40510031e-03   0.00000000e+00  -1.04648185e-02
  -7.40477074e-03  -1.04753805e-02   0.00000000e+00   7.40510031e-03
   0.00000000e+00   1.04648185e-02  -7.40477074e-03]
Gradient in Internal coordinates.
[  1.28284490e-02   1.28196350e-02   1.28284490e-02   1.28196350e-02
   7.32160379e-07   3.83522496e-06   7.32160379e-07   7.32160379e-07
  -6.76417342e-06   7.32160379e-07]


Lets try the reverse transformation and see if we return to the original gradient.

In [6]:
gx2 = np.dot(B.T,gq)
print ("Gradient in Cartesian coordinates (from internals).")
# Removing some noise
gx2[:] = [0 if fabs(x) < 1.0e-16 else x for x in gx2]
print ( gx2 )


Gradient in Cartesian coordinates (from internals).
[  0.00000000e+00   0.00000000e+00  -6.59135991e-07   1.04753805e-02
   0.00000000e+00   7.40510031e-03   0.00000000e+00  -1.04648185e-02
  -7.40477074e-03  -1.04753805e-02   0.00000000e+00   7.40510031e-03
   0.00000000e+00   1.04648185e-02  -7.40477074e-03]


For information on the transformations of energy second derivatives, see the separate tutorial file on Hessians.